In [1]:
import numpy as np

In [2]:
W = 1   #Width 
K = 1   #Depth
n = 5
dimensions = 3
def atomic_distances(pos, output, L=1, W=1): # output = 0 gives the relative distances, output = 1 gives the relative positions
    rel_pos = pos[:, None, :] - pos[None, :, :]                 # returns one matrix for each particle. Relative distances within the box
    rel_dist = np.zeros((n, n, 1))
    for i in range(n):
        for k in range(n):
            for l in range(dimensions):
                if l == 0: # x-values i.e. change by L
                    rel_pos[i,k,l] = min(rel_pos[i,k,l], (rel_pos[i,k,l]+L)%L, (rel_pos[i,k,l]-L)%L, rel_pos[i,k,l]+L, rel_pos[i,k,l]-L, key=abs) # takes the smallest distance comparing all images
                if l == 1:
                    rel_pos[i,k,l] = min(rel_pos[i,k,l], (rel_pos[i,k,l]+W)%W, (rel_pos[i,k,l]-W)%W, rel_pos[i,k,l]+W, rel_pos[i,k,l]-W, key=abs)
                if l == 2:
                    rel_pos[i,k,l] = min(rel_pos[i,k,l], (rel_pos[i,k,l]+K)%K, (rel_pos[i,k,l]-K)%K, rel_pos[i,k,l]+K, rel_pos[i,k,l]-K, key=abs)
    for i in range(n):    
        for k in range(n):
            rel_dist[i,k] = np.sqrt(np.dot(rel_pos[i,k,:],rel_pos[i,k,:]))
    if output == 0:
        return rel_dist
    elif output == 1:
        return rel_pos
    else:
        print("The second parameter in the lj_forces must be either 0(for distances) or 1(for positions)")

In [3]:
#function to compute the potential energy
def pot_en(position): #position is the matrix with all the positions stored in it
    current_pos = np.copy(position[:,-3:]) #nx3 matrix with the positions at the last time step
    rel_pos = atomic_distances(current_pos, 1) # with pos = np.array(n, 3)
    r = np.sqrt(np.sum((rel_pos)**2, axis=2)) #n x n simmetric matrix, r[i,j] is the distance between the i-th and the j-th particles
    i = np.where(r == 0) # indices where the r matrix is 0
    ones = np.zeros((n,n)) # nxn matrix
    ones[i] = 1 # matrix with ones where r has zeros
    R = r + ones  # same matrix as r, but with ones where r has zeros. we do this to avoid dividing by zero
    u = 4*(1/R**12 - 1/R**6)
    U = np.sum(u)
    return U

In [4]:
A = np.array([[0.26,0.37,0.81],[0.45,0.62,0.94],[0.75,0.53,0.38],[0.54,0.38,0.82],[0.13,0.26,0.87]])
pot_en(A)

6732359651.986205